In [4]:
SHP_FILE = '../data/inputs/2308.gpkg'
DATA_DIR = "../tmpdata"
SRC_DIR = "~/projects/afg-clustering/src"
N_CORES = 3

In [5]:
import geopandas as gpd
from pathlib import Path
import os
import shutil
import subprocess

In [6]:
input_ = gpd.read_file(SHP_FILE)


In [7]:
# TMP_DATA_DIR = f"{DATA_DIR}/dnq"
# if os.path.exists(TMP_DATA_DIR):
#     shutil.rmtree(TMP_DATA_DIR)
# Path(TMP_DATA_DIR).mkdir(parents=True, exist_ok=True)

In [10]:
INTERIM_DIR = f'{DATA_DIR}/interim'
Path(INTERIM_DIR).mkdir(parents=True, exist_ok=True)
sh = ShapefileHelper(SHP_FILE, INTERIM_DIR)
sh.make_grid(50000, "grid");

--------- Generating grid at 50000mx50000m ...
--------- Grid generation complete. Saving to disk...
--------- Successfully saved to disk: ../tmpdata/interim/grid.gpkg


In [11]:
GRID_PATH = f"{INTERIM_DIR}/grid.gpkg"
grid = gpd.read_file(GRID_PATH)

In [12]:
for i, tile in grid.iterrows():
    temp = gpd.GeoDataFrame(grid.iloc[i]).transpose()
    temp = temp[['grid_id', 'geometry']]
    temp['grid_id'] = temp['grid_id'].astype('int')
    print(temp.columns)
    temp.to_file(f'{INTERIM_DIR}/{temp.iloc[0][0]}.gpkg')

Index(['grid_id', 'geometry'], dtype='object')
Index(['grid_id', 'geometry'], dtype='object')
Index(['grid_id', 'geometry'], dtype='object')
Index(['grid_id', 'geometry'], dtype='object')


In [45]:
grids = [tile for tile in os.listdir(INTERIM_DIR) if tile != "grid.gpkg" and  tile != ".ipynb_checkpoints"]

In [47]:
DNQ_DIR = f"{INTERIM_DIR}/dnq"
if os.path.exists(DNQ_DIR):
    shutil.rmtree(DNQ_DIR)
# Path(DNQ_DIR).mkdir(parents=True, exist_ok=True)
for grid in grids:
    temp = grid.split(".gpkg")[0]
    Path(f"{DNQ_DIR}/{temp}/inputs").mkdir(parents=True, exist_ok=True)
    shutil.copy2(f"{DATA_DIR}/inputs/2019_E060N40_PROBAV_LC100_global_v3.0.1_2019.tif",f"{DNQ_DIR}/{temp}/inputs")

In [48]:
grids = [f"{INTERIM_DIR}/{tile}" for tile in grids]

In [59]:
for tile in grids:
    TILE = tile.split("/")[-1].split(".gpkg")[0]
    TMP_DATA_DIR = f"{DNQ_DIR}/{TILE}"
    TMP_INTERIM_DIR = f"{TMP_DATA_DIR}/interim"
    Path(f"{TMP_INTERIM_DIR}/{temp}/inputs").mkdir(parents=True, exist_ok=True)
    
    pre = open(f"{TMP_INTERIM_DIR}/preprocess.out", "w")
    fix = open(f"{TMP_INTERIM_DIR}/fix.out", "w")
    pred = open(f"{TMP_INTERIM_DIR}/predict.out", "w")

    subprocess.call(f"python -u {SRC_DIR}/poppy-preprocess.py {TMP_DATA_DIR} {tile} --upto_step download --year 2019 --n_cores 5", shell=True, stdout=pre)
    subprocess.call(f"python -u {SRC_DIR}/poppy-fix-downloads.py {TMP_DATA_DIR} --n_cores 5", shell=True, stdout=fix)
    subprocess.call(f"python -u {SRC_DIR}/poppy-predict.py {TMP_DATA_DIR} ${tile} /data/tmp/arogya/data/outputs/models/kmeans_3_diff_bands --model_type kmeans --num 3 --name kmeans_3", shell=True, stdout=pred)
    

KeyboardInterrupt: 

In [ ]:
import argparse

parser = argparse.ArgumentParser()

parser.add_argument("data_dir", help="What dir should I use for generating data")
parser.add_argument("shp", help="Shapefile")

args = parser.parse_args()

DATA_DIR = args.data_dir
SHP = args.shp


from utils.shapefile import ShapefileHelper
import geopandas as gpd
from pathlib import Path
import os
import shutil

INTERIM_DIR = f'{DATA_DIR}/interim'
Path(INTERIM_DIR).mkdir(parents=True, exist_ok=True)
sh = ShapefileHelper(SHP_FILE, INTERIM_DIR)
sh.make_grid(50000, "grid");

GRID_PATH = f"{INTERIM_DIR}/grid.gpkg"
grid = gpd.read_file(GRID_PATH)

for i, tile in grid.iterrows():
    temp = gpd.GeoDataFrame(grid.iloc[i]).transpose()
    temp = temp[['grid_id', 'geometry']]
    temp['grid_id'] = temp['grid_id'].astype('int')
    temp.to_file(f'{INTERIM_DIR}/{temp.iloc[0][0]}.gpkg')
    
grids = [tile for tile in os.listdir(INTERIM_DIR) if tile != "grid.gpkg" and  tile != ".ipynb_checkpoints"]

DNQ_DIR = f"{INTERIM_DIR}/dnq"
if os.path.exists(DNQ_DIR):
    shutil.rmtree(DNQ_DIR)
for grid in grids:
    temp = grid.split(".gpkg")[0]
    Path(f"{DNQ_DIR}/{temp}/inputs").mkdir(parents=True, exist_ok=True)
    shutil.copy2(f"{DATA_DIR}/inputs/2019_E060N40_PROBAV_LC100_global_v3.0.1_2019.tif",f"{DNQ_DIR}/{temp}/inputs")

In [9]:
import geopandas as gpd
from osgeo import ogr
from math import ceil
import shutil
import os
from shapely.geometry import box
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")


# Shape file helper class
class ShapefileHelper():
    
    def __init__(self, dissolved_aoi_shapefile, vector_output_dir):
        self.raw = dissolved_aoi_shapefile
        self.vector_output_dir = vector_output_dir + "/"
        self._make_dir()
        self._read_file()
    
    def make_grid(self, resolution, name, out_crs="epsg:4326", id_col="grid_id"):
#         self.gdf.boundary.plot()
        
        gridWidth, gridHeight = resolution, resolution
        xmin, ymin, xmax, ymax = self.gdf.bounds.iloc[0]['minx'],  self.gdf.bounds.iloc[0]['miny'],  self.gdf.bounds.iloc[0]['maxx'],  self.gdf.bounds.iloc[0]['maxy']
        
        outputGridfn = name
        
        # get rows
        rows = ceil((ymax-ymin)/gridHeight)
        # get columns
        cols = ceil((xmax-xmin)/gridWidth)

        # start grid cell envelope
        ringXleftOrigin = xmin
        ringXrightOrigin = xmin + gridWidth
        ringYtopOrigin = ymax
        ringYbottomOrigin = ymax-gridHeight
        
        # create output file
        outDriver = ogr.GetDriverByName('ESRI Shapefile')
#         if os.path.exists(self.vector_output_dir + outputGridfn):
#             print("Deleting pre-existing shapefile: {}{}{}.shp".format(self.vector_output_dir,outputGridfn+"/", outputGridfn))
#             shutil.rmtree(self.vector_output_dir + outputGridfn,  ignore_errors=True)
            
#         if os.path.isdir(self.vector_output_dir):
#             print("Deleting pre-existing folder: {}{}/".format(self.vector_output_dir, outputGridfn))
#             shutil.rmtree(self.vector_output_dir + outputGridfn,  ignore_errors=True)
#         else:
#             os.makedirs(self.vector_output_dir)
#             print("Created new directory: {}{}/".format(self.vector_output_dir, outputGridfn))
            
            
        outDataSource = outDriver.CreateDataSource(self.vector_output_dir + outputGridfn + ".shp")
        outLayer = outDataSource.CreateLayer(outputGridfn,geom_type=ogr.wkbPolygon)
        featureDefn = outLayer.GetLayerDefn()
        
        print(f'--------- Generating grid at {resolution}mx{resolution}m ...')

        # create grid cells
        countcols = 0
        while countcols < cols:
            countcols += 1

            # reset envelope for rows
            ringYtop = ringYtopOrigin
            ringYbottom =ringYbottomOrigin
            countrows = 0

            while countrows < rows:
                countrows += 1
                ring = ogr.Geometry(ogr.wkbLinearRing)
                ring.AddPoint(ringXleftOrigin, ringYtop)
                ring.AddPoint(ringXrightOrigin, ringYtop)
                ring.AddPoint(ringXrightOrigin, ringYbottom)
                ring.AddPoint(ringXleftOrigin, ringYbottom)
                ring.AddPoint(ringXleftOrigin, ringYtop)
                poly = ogr.Geometry(ogr.wkbPolygon)
                poly.AddGeometry(ring)

                # add new geom to layer
                outFeature = ogr.Feature(featureDefn)
                outFeature.SetGeometry(poly)
                outLayer.CreateFeature(outFeature)
                outFeature.Destroy

                # new envelope for next poly
                ringYtop = ringYtop - gridHeight
                ringYbottom = ringYbottom - gridHeight

            # new envelope for next poly
            ringXleftOrigin = ringXleftOrigin + gridWidth
            ringXrightOrigin = ringXrightOrigin + gridWidth
        # Close DataSources
        outDataSource.Destroy()
        print('--------- Grid generation complete. Saving to disk...')        
        
        
#         # clip to boundary
        output = gpd.read_file(self.vector_output_dir + outputGridfn +".shp")
        
        output = output.set_crs("epsg:32642").to_crs(out_crs).reset_index().rename(columns={"index": id_col}).drop('FID', axis=1)
        output = gpd.sjoin(output, self.gdf.to_crs("epsg:4326")).drop('index_right', axis=1)
        
        
#         output = gpd.sjoin(output.to_crs("epsg:4326"), self.gdf).drop('index_right', axis=1)

        
#         print("Saving generated shapefile to disk..")
#         if os.path.exists(self.vector_output_dir + outputGridfn):
#             directory = self.vector_output_dir + outputGridfn
#             for f in os.listdir(directory):
#                 os.remove(os.path.join(directory, f))

        self.output = output
        if id_col in output.columns:  
            output = output.drop(id_col, axis=1).reset_index().drop('index', axis=1).reset_index().rename(columns={'index': id_col})
        else:
            output = output.reset_index().drop('index', axis=1).reset_index().rename(columns={'index': id_col})
        output.to_file(self.vector_output_dir + outputGridfn + ".gpkg", driver='GPKG')
        self._clean_dir()
        print("--------- Successfully saved to disk: {}".format(self.vector_output_dir + outputGridfn+ ".gpkg"))
        return output

    def subset_grid(self, grid_path, aoi_path):
        aoi = gpd.read_file(aoi_path)
        grid = gpd.read_file(grid_path)
        
        subset = gpd.sjoin(grid, aoi).drop('index_right', axis=1)
        subset.to_file(grid_path, driver='GPKG')
        
    def _read_file(self):
        raw = gpd.read_file(self.raw)
#         geom = box(*raw.total_bounds)
#         gdf = gpd.GeoDataFrame([{"geometry": geom}])
#         gdf = gdf.set_crs("epsg:4326")
        self.gdf = raw.to_crs("epsg:32642")
    
    def _make_dir(self):
        Path(self.vector_output_dir).mkdir(parents=True, exist_ok=True)
    
    def _clean_dir(self):
        test = os.listdir(self.vector_output_dir)

        for item in test:
            if item.endswith(".shp"):
                os.remove(os.path.join(self.vector_output_dir, item))
            if item.endswith(".shx"):
                os.remove(os.path.join(self.vector_output_dir, item))
            if item.endswith(".dbf"):
                os.remove(os.path.join(self.vector_output_dir, item))
        
